In [2]:
import pandas as pd
pd.options.display.max_columns = None
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog

import numpy as np

In [3]:

# Get all NBA teams
nba_teams = teams.get_teams()
nba_teams[0]

{'id': 1610612737,
 'full_name': 'Atlanta Hawks',
 'abbreviation': 'ATL',
 'nickname': 'Hawks',
 'city': 'Atlanta',
 'state': 'Georgia',
 'year_founded': 1949}

In [4]:
pd.DataFrame(nba_teams).to_csv("../data/team_info.csv", index=False)

In [5]:
# # Select a team
# team = [team for team in nba_teams if team['abbreviation'] == 'ATL'][0]

# # Get the team ID
# team_id = team['id']

# # Query the API
# response = teamgamelog.TeamGameLog(
#     team_id=team_id,
#     season='2012-13',  # 2020-2021 season
#     season_type_all_star='Regular Season'  # Regular season
# )

# df = response.get_data_frames()[0].sort_values(by="Game_ID")
# print(df.shape)
# df.tail()

In [6]:
# Get all teams
nba_teams = teams.get_teams()

# Initialize a list to store all games dataframes
all_games = []

season_start = 2013
num_seasons = 10
for season in range(season_start, season_start + num_seasons):
    print(season)

    # Loop over all teams
    for team in nba_teams[:]:
        team_id = team["id"]
        # print(team["full_name"], )

        # Query the API
        response = teamgamelog.TeamGameLog(
            team_id=team_id,
            season=season,
            season_type_all_star='Regular Season'
        )

        games = response.get_data_frames()[0].sort_values(by="Game_ID")
        games['Team'] = team['full_name']
        games['SeasonStart'] = season
        # print(f"num games: {games.shape[0]}")
        
        # Append games dataframe to the list
        all_games.append(games)

    # Concatenate all games dataframes
    all_games_df = pd.concat(all_games)
    

print(all_games_df.shape)
all_games_df

Atlanta Hawks
Boston Celtics
Cleveland Cavaliers
New Orleans Pelicans
Chicago Bulls
Dallas Mavericks
Denver Nuggets
Golden State Warriors
Houston Rockets
Los Angeles Clippers
Los Angeles Lakers
Miami Heat
Milwaukee Bucks
Minnesota Timberwolves
Brooklyn Nets
New York Knicks
Orlando Magic
Indiana Pacers
Philadelphia 76ers
Phoenix Suns
Portland Trail Blazers
Sacramento Kings
San Antonio Spurs
Oklahoma City Thunder
Toronto Raptors
Utah Jazz
Memphis Grizzlies
Washington Wizards
Detroit Pistons
Charlotte Hornets
Atlanta Hawks
Boston Celtics
Cleveland Cavaliers
New Orleans Pelicans
Chicago Bulls
Dallas Mavericks
Denver Nuggets
Golden State Warriors
Houston Rockets
Los Angeles Clippers
Los Angeles Lakers
Miami Heat
Milwaukee Bucks
Minnesota Timberwolves
Brooklyn Nets
New York Knicks
Orlando Magic
Indiana Pacers
Philadelphia 76ers
Phoenix Suns
Portland Trail Blazers
Sacramento Kings
San Antonio Spurs
Oklahoma City Thunder
Toronto Raptors
Utah Jazz
Memphis Grizzlies
Washington Wizards
Detroit Pi

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,Team,SeasonStart
81,1610612737,0021300012,"OCT 30, 2013",ATL @ DAL,L,0,1,0.000,240,37,76,0.487,8,24,0.333,27,35,0.771,5,28,33,27,16,5,17,20,109,Atlanta Hawks,2013
80,1610612737,0021300023,"NOV 01, 2013",ATL vs. TOR,W,1,1,0.500,240,36,77,0.468,10,23,0.435,20,31,0.645,7,30,37,24,7,3,13,12,102,Atlanta Hawks,2013
79,1610612737,0021300046,"NOV 03, 2013",ATL @ LAL,L,1,2,0.333,240,40,90,0.444,9,22,0.409,14,26,0.538,10,42,52,26,9,3,11,22,103,Atlanta Hawks,2013
78,1610612737,0021300059,"NOV 05, 2013",ATL @ SAC,W,2,2,0.500,240,39,81,0.481,8,22,0.364,19,26,0.731,9,27,36,26,11,7,14,15,105,Atlanta Hawks,2013
77,1610612737,0021300071,"NOV 07, 2013",ATL @ DEN,L,2,3,0.400,240,43,94,0.457,8,26,0.308,13,19,0.684,13,33,46,31,11,9,11,23,107,Atlanta Hawks,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,1610612766,0022201148,"MAR 31, 2023",CHA vs. CHI,L,26,52,0.333,240,35,77,0.455,14,37,0.378,7,10,0.700,4,32,36,27,3,5,21,21,91,Charlotte Hornets,2022
3,1610612766,0022201163,"APR 02, 2023",CHA vs. TOR,L,26,53,0.329,240,42,85,0.494,15,31,0.484,9,13,0.692,10,27,37,26,3,4,17,11,108,Charlotte Hornets,2022
2,1610612766,0022201176,"APR 04, 2023",CHA vs. TOR,L,26,54,0.325,240,36,77,0.468,12,31,0.387,16,19,0.842,6,36,42,27,3,4,17,22,100,Charlotte Hornets,2022
1,1610612766,0022201202,"APR 07, 2023",CHA vs. HOU,L,26,55,0.321,240,38,88,0.432,13,41,0.317,20,27,0.741,7,34,41,22,10,9,17,21,109,Charlotte Hornets,2022


In [7]:
# opponents abbreviation
all_games_df['opponent_abbreviation'] = all_games_df['MATCHUP'].str[-3:]
all_games_df['team_abbreviation'] = all_games_df['MATCHUP'].str[:3]

# home or away
all_games_df['is_home_game'] = np.where(all_games_df['MATCHUP'].str.contains("@"), 0, 1)

# convert to date
all_games_df['GAME_DATE'] = pd.to_datetime(all_games_df['GAME_DATE'])

all_games_df.head()

/var/folders/dq/n1gz5xz55cj7qv8f04fcsb0m0000gp/T/ipykernel_70115/2350903109.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_games_df['GAME_DATE'] = pd.to_datetime(all_games_df['GAME_DATE'])


,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,Team,SeasonStart,opponent_abbreviation,team_abbreviation,is_home_game
81,1610612737,0021300012,2013-10-30,ATL @ DAL,L,0,1,0.000,240,37,76,0.487,8,24,0.333,27,35,0.771,5,28,33,27,16,5,17,20,109,Atlanta Hawks,2013,DAL,ATL,0
80,1610612737,0021300023,2013-11-01,ATL vs. TOR,W,1,1,0.500,240,36,77,0.468,10,23,0.435,20,31,0.645,7,30,37,24,7,3,13,12,102,Atlanta Hawks,2013,TOR,ATL,1
79,1610612737,0021300046,2013-11-03,ATL @ LAL,L,1,2,0.333,240,40,90,0.444,9,22,0.409,14,26,0.538,10,42,52,26,9,3,11,22,103,Atlanta Hawks,2013,LAL,ATL,0
78,1610612737,0021300059,2013-11-05,ATL @ SAC,W,2,2,0.500,240,39,81,0.481,8,22,0.364,19,26,0.731,9,27,36,26,11,7,14,15,105,Atlanta Hawks,2013,SAC,ATL,0
77,1610612737,0021300071,2013-11-07,ATL @ DEN,L,2,3,0.400,240,43,94,0.457,8,26,0.308,13,19,0.684,13,33,46,31,11,9,11,23,107,Atlanta Hawks,2013,DEN,ATL,0


In [8]:
print(all_games_df.shape)
all_games_df

(23958, 32)


,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,Team,SeasonStart,opponent_abbreviation,team_abbreviation,is_home_game
81,1610612737,0021300012,2013-10-30,ATL @ DAL,L,0,1,0.000,240,37,76,0.487,8,24,0.333,27,35,0.771,5,28,33,27,16,5,17,20,109,Atlanta Hawks,2013,DAL,ATL,0
80,1610612737,0021300023,2013-11-01,ATL vs. TOR,W,1,1,0.500,240,36,77,0.468,10,23,0.435,20,31,0.645,7,30,37,24,7,3,13,12,102,Atlanta Hawks,2013,TOR,ATL,1
79,1610612737,0021300046,2013-11-03,ATL @ LAL,L,1,2,0.333,240,40,90,0.444,9,22,0.409,14,26,0.538,10,42,52,26,9,3,11,22,103,Atlanta Hawks,2013,LAL,ATL,0
78,1610612737,0021300059,2013-11-05,ATL @ SAC,W,2,2,0.500,240,39,81,0.481,8,22,0.364,19,26,0.731,9,27,36,26,11,7,14,15,105,Atlanta Hawks,2013,SAC,ATL,0
77,1610612737,0021300071,2013-11-07,ATL @ DEN,L,2,3,0.400,240,43,94,0.457,8,26,0.308,13,19,0.684,13,33,46,31,11,9,11,23,107,Atlanta Hawks,2013,DEN,ATL,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,1610612766,0022201148,2023-03-31,CHA vs. CHI,L,26,52,0.333,240,35,77,0.455,14,37,0.378,7,10,0.700,4,32,36,27,3,5,21,21,91,Charlotte Hornets,2022,CHI,CHA,1
3,1610612766,0022201163,2023-04-02,CHA vs. TOR,L,26,53,0.329,240,42,85,0.494,15,31,0.484,9,13,0.692,10,27,37,26,3,4,17,11,108,Charlotte Hornets,2022,TOR,CHA,1
2,1610612766,0022201176,2023-04-04,CHA vs. TOR,L,26,54,0.325,240,36,77,0.468,12,31,0.387,16,19,0.842,6,36,42,27,3,4,17,22,100,Charlotte Hornets,2022,TOR,CHA,1
1,1610612766,0022201202,2023-04-07,CHA vs. HOU,L,26,55,0.321,240,38,88,0.432,13,41,0.317,20,27,0.741,7,34,41,22,10,9,17,21,109,Charlotte Hornets,2022,HOU,CHA,1


In [9]:
# how many games per team per regular season
piv = all_games_df.pivot_table(
    index=["Team", "SeasonStart"],
    values=["Game_ID"],
    aggfunc='count'
)
print(piv.shape)
piv.head(30)

(300, 1)


Game_ID
Team           SeasonStart         
Atlanta Hawks  2013              82
               2014              82
               2015              82
               2016              82
               2017              82
               2018              82
               2019              67
               2020              72
               2021              82
               2022              82
Boston Celtics 2013              82
               2014              82
               2015              82
               2016              82
               2017              82
               2018              82
               2019              72
               2020              72
               2021              82
               2022              82
Brooklyn Nets  2013              82
               2014              82
               2015              82
               2016              82
               2017              82
               2018              82
               2019              72
               2020              72
               2021              82
               2022              82

In [10]:
# why does 2020 only have 72 games

In [11]:
import time
print(time.ctime())
all_games_df.to_csv("../data/2013-2022-regular-season-matchups.csv", index=False)


Fri Aug 18 21:05:46 2023
